https://www.kaggle.com/code/kareemellithy/imdb-50k-movies-reviews-lstm/notebook

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

## Carreguem el dataset en un dataframe

In [2]:
df = pd.read_csv("IMDB Dataset.csv")
df

In [3]:
df.info()

In [4]:
df.shape

In [5]:
df['sentiment'].value_counts().reset_index()

## Netejem el dataframe

Eliminem els caràcters innecessaris, etiquetes html, símbols, etc

In [6]:
def remove_numbers(text):
    return re.sub('[0-9]+', '', text)
def remove_html_tags(text):
    return re.sub(r'<[^>]+>', '', text)
def remove_symbols(text):
    return re.sub("[!@#$%^&*(){}£\/'']",'',text)

df['review']= df['review'].apply(lambda x: remove_numbers(x))
df['review']= df['review'].apply(lambda x: remove_html_tags(x))
df['review']= df['review'].apply(lambda x: remove_symbols(x))

In [7]:
print(f"{df['review'][9000]} -> {df['sentiment'][9000]}")

In [8]:
txt = df['review'][9862]
txt


In [9]:
for token in txt:
    print(token)

In [10]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
df['sentiment']=label.fit_transform(df['sentiment'])
df

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'],test_size=0.25, random_state=42, stratify=df['sentiment'])

In [12]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

In [13]:
v = TfidfVectorizer()

X_train_cv = v.fit_transform(X_train)
X_test_cv = v.transform(X_test)

In [14]:
RFC_model = RandomForestClassifier()

RFC_model.fit(X_train_cv, y_train)

In [15]:
y_pred = RFC_model.predict(X_test_cv)

In [16]:
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))